In [41]:
# Import libraries yang dibutuhkan
from bs4 import BeautifulSoup
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.common.exceptions import NoSuchElementException, TimeoutException
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import time

In [42]:
# Inisialisasi WebDriver
options = webdriver.ChromeOptions()
options.add_argument("start-maximized")
url = "https://www.carsome.id/en/buy-car" # URL target yang akan discraping
driver = webdriver.Chrome(options=options)
driver.get(url)

In [43]:
# Menentukan CSS selector untuk pop-up dan tombol close
popup_selector = "div.location-model-box"
close_button_selector = "img.location-model-box__head-close"

# Tunggu hingga pop-up muncul dan temukan tombol close
try:
    popup = WebDriverWait(driver, 10).until(EC.visibility_of_element_located((By.CSS_SELECTOR, popup_selector)))
    close_button = popup.find_element(By.CSS_SELECTOR, close_button_selector)
    close_button.click()  # Klik tombol close
except (NoSuchElementException, TimeoutException):
    pass  # Jika pop-up tidak ditemukan, lanjutkan scraping

In [44]:
# Fungsi untuk mengekstrak informasi mobil dari setiap item
def extract_car_info(item):
    car_info = {}

    title_elem = item.select_one('.mod-b-card__title')
    if title_elem:
        # Mendapatkan teks dari elemen '.mod-b-card__title'
        title_text = title_elem.text.strip()
        
        # Memecah teks judul berdasarkan karakter newline
        title_parts = title_text.split('\n')
        
        # Mendapatkan merek (brand) dari bagian judul
        brand = title_parts[1].strip()  # Mendapatkan merek dari bagian kedua
        car_info['brand'] = brand
        
        # Mendapatkan model dari bagian judul
        model = ' '.join(title_parts[2:])  # Menggabungkan semua elemen kecuali yang pertama (tahun)
        car_info['model'] = model.strip()   # Menghapus spasi ekstra
        
        # Mendapatkan tahun dari bagian judul
        year = title_parts[0].strip()  # Mengambil tahun dari bagian pertama
        car_info['year'] = year
    else:
        car_info['year'] = 'N/A'
        car_info['model'] = 'N/A'

    # Mendapatkan mileage dari elemen '.mod-b-card__car-other'
    mileage_elem = item.select_one('.mod-b-card__car-other span:nth-of-type(1)')
    if mileage_elem:
        car_info['mileage'] = mileage_elem.text.strip()
    else:
        car_info['mileage'] = 'N/A'

    # Mendapatkan jenis transmisi dari elemen '.mod-b-card__car-other'
    transmission_elem = item.select_one('.mod-b-card__car-other span:nth-of-type(2)')
    if transmission_elem:
        car_info['transmission'] = transmission_elem.text.strip()
    else:
        car_info['transmission'] = 'N/A'

    # Mendapatkan lokasi seller dari elemen '.mod-b-card__car-other'
    location_elem = item.select_one('.mod-b-card__car-other span:nth-of-type(3)')
    if location_elem:
        car_info['seller_location'] = location_elem.text.strip()
    else:
        car_info['seller_location'] = 'N/A'

    # Mendapatkan harga dari elemen '.mod-card__price-cash'
    price_elem = item.select_one('.mod-card__price-cash')
    if price_elem:
        car_info['price'] = price_elem.text.strip()
    else:
        car_info['price'] = 'N/A'

    # Mendapatkan cicilan dari elemen '.mod-tooltipMonthPay'
    installment = item.select_one('.mod-tooltipMonthPay')
    if installment:
        car_info['installment'] = installment.text.strip()
    else: 
        car_info['installment'] = 'N/A'
        
    return car_info # Mengembalikan hasil ekstrasi sebagai sebuah dictionary

In [45]:
# Looping untuk melalukan scraping di beberapa halaman
products = []
while True:
    time.sleep(10)  # Tunggu 10 detik sebelum melanjutkan ke halaman berikutnya (bisa disesuaikan sesuai kecepatan internet)
    try:
        next_button = driver.find_element(By.CSS_SELECTOR, "button.v-pagination__navigation[aria-label='Next page']")
        if next_button.is_enabled():
            next_button.click()  # Klik tombol halaman berikutnya
            time.sleep(10)  # Tunggu halaman dimuat sepenuhnya
        else:
            break  # Keluar dari loop jika tidak ada lagi halaman berikutnya
    except NoSuchElementException:
        break  # Keluar dari loop jika tidak ada tombol navigasi
    except TimeoutException:
        break  # Keluar dari loop jika waktu tunggu sudah habis
    
    # Parsing HTML dari halaman yang baru dimuat
    soup = BeautifulSoup(driver.page_source, "html.parser")
    # Ekstrak informasi mobil dari setiap item
    for item in soup.select('div.list-card__item'):
        car_info = extract_car_info(item)
        products.append(car_info)
        
# Tutup WebDriver
driver.quit()

In [50]:
# Mengubah hasil scrapping menjadi dataframe
carsome_scrapping_df = pd.DataFrame(products)

carsome_scrapping_df

,brand,model,year,mileage,transmission,seller_location,price,installment
0,Daihatsu,AYLA R 1.2,2020,54.801 km,Automatic,Tangerang Selatan,Rp 137.000.000 (Cash),"Rp 3,17 jt/mo"
1,Toyota,YARIS G 1.5,2019,65.003 km,Automatic,Tangerang Selatan,Rp 189.000.000 (Cash),"Rp 4,35 jt/mo"
2,Daihatsu,TERIOS X 1.5,2015,115.866 km,Manual,Tangerang Selatan,Rp 140.000.000 (Cash),"Rp 3,24 jt/mo"
3,Daihatsu,AYLA R 1.2,2019,57.138 km,Manual,Tangerang Selatan,Rp 112.000.000 (Cash),"Rp 2,62 jt/mo"
4,Honda,CR-V TURBO 1.5,2017,117.511 km,Automatic,Tangerang Selatan,Rp 317.000.000 (Cash),"Rp 7,22 jt/mo"
...,...,...,...,...,...,...,...,...
133,NaN,N/A,N/A,N/A,N/A,N/A,N/A,N/A
134,Toyota,AVANZA G 1.3,2019,58.385 km,Manual,Tangerang Selatan,Rp 180.000.000 (Cash),"Rp 4,14 jt/mo"
135,Honda,BRIO RS 1.2,2018,44.339 km,Automatic,Bekasi,Rp 155.500.000 (Cash),"Rp 3,59 jt/mo"
136,Nissan,GRAND LIVINA HIGHWAY STAR AUTECH ...,2017,117.783 km,Automatic,Bekasi,Rp 152.500.000 (Cash),"Rp 3,52 jt/mo"
